<a href="https://colab.research.google.com/github/Shazaam91/Hate-Speech-Detection-/blob/BiLSTM/Hate_Speech_Sinhala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#import required libraries
import tensorflow as tf
import csv
import random
import numpy as np
import pandas as pd
import io
import gzip

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2


In [46]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
#install fasttext
! pip install fasttext

In [48]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.bin.gz

--2024-08-18 19:42:27--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.74.45, 13.227.74.118, 13.227.74.9, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.74.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 481477801 (459M) [binary/octet-stream]
Saving to: ‘cc.si.300.vec.gz.7’

cc.si.300.vec.gz.7  100%[===================>] 459.17M  38.5MB/s    in 12s     

2024-08-18 19:42:39 (39.1 MB/s) - ‘cc.si.300.vec.gz.7’ saved [481477801/481477801]

--2024-08-18 19:42:39--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.74.45, 13.227.74.118, 13.227.74.9, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.74.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3181346570 (3.0G) [application/octet-stream]
Saving to: ‘cc

In [49]:
import os
import gzip
import shutil

# Path to the gzip file
gzip_file_path = '/content/cc.si.300.bin.gz'
# Path where the extracted file will be saved
extracted_file_path = '/content/cc.si.300.bin'

# Remove the file if it already exists to avoid overwrite prompt
if os.path.exists(extracted_file_path):
    os.remove(extracted_file_path)

# Extract the gzip file
with gzip.open(gzip_file_path, 'rb') as f_in:
    with open(extracted_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [50]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!sudo python setup.py install


fatal: destination path 'fastText' already exists and is not an empty directory.
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [51]:
import fasttext
import fasttext.util

ft = fasttext.load_model('/content/cc.si.300.bin')
ft.get_dimension()

300

In [52]:
fasttext.util.reduce_model(ft, 100)

In [53]:
# Reading the dataset
dataset = []
error_count = 0  # To count errors

In [54]:
import csv

with open("/content/drive/MyDrive/sinhala-hate-speech-dataset.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    header = reader.fieldnames
    print("Header Row:", header)


Header Row: ['id', 'comment', 'label']


In [55]:
# Reading the dataset using the correct column names
dataset = []
error_count = 0  # To count errors

with open("/content/drive/MyDrive/sinhala-hate-speech-dataset.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        post = row['comment']  # Correct column name for the post
        label = row['label']  # Correct column name for the label

        # Skip rows with empty post or label
        if not post or not label:
            print(f"Skipping row {i}: missing post or label")
            continue

        item = [post]

        if label == '0':
            item.append(0)
        elif label == '1':
            item.append(1)
        else:
            print(f"ERROR at row {i}: Unexpected label {label}")
            error_count += 1
            continue

        dataset.append(item)

In [56]:
# Final counts
print(f"Total rows processed: {i + 1}")
print(f"Total valid entries: {len(dataset)}")
print(f"Total errors: {error_count}")



Total rows processed: 6345
Total valid entries: 6345
Total errors: 0


In [57]:
# Checking the lengths of posts and labels
posts = [item[0] for item in dataset]
labels = [item[1] for item in dataset]

print(f"Length of posts: {len(posts)}")
print(f"Length of labels: {len(labels)}")

Length of posts: 6345
Length of labels: 6345


In [58]:
# Separate posts and labels
posts = [item[0] for item in dataset]
labels = [item[1] for item in dataset]

# Check the lengths to ensure they match
print(f"Length of posts: {len(posts)}")
print(f"Length of labels: {len(labels)}")

Length of posts: 6345
Length of labels: 6345


In [59]:
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize posts before resampling
tokenizer = Tokenizer()
tokenizer.fit_on_texts(posts)
sequences = tokenizer.texts_to_sequences(posts)
padded_sequences = pad_sequences(sequences, maxlen=120)

# Convert labels to a NumPy array for SMOTE
labels = np.array(labels)

# Apply SMOTE to balance the dataset
sm = SMOTE(random_state=42)
padded_sequences, labels = sm.fit_resample(padded_sequences, labels)

In [60]:
from sklearn.model_selection import train_test_split
import numpy as np

# Split the data into training and testing sets
padded_sequences_train, padded_sequences_test, labels_train, labels_test = train_test_split(
    np.asarray(padded_sequences),
    np.asarray(labels),
    test_size=0.1,
    random_state=42
)


In [61]:
# Reusing the tokenizer from the previous step, so no need to redefine it
vocab_size = len(tokenizer.word_index) + 1  # Ensure that vocab_size includes all unique words plus one for padding

embedding_dim = 100
max_length = 120
trunc_type = 'post'
oov_tok = ""

# The padded_sequences_train and padded_sequences_test are already prepared, so no need to re-tokenize
padded = padded_sequences_train
testing_padded = padded_sequences_test


In [62]:
# Define word_index to avoid NameError
word_index = tokenizer.word_index

# Create embeddings_matrix
embeddings_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i < vocab_size:  # Ensure the index is within the vocabulary size
        try:
            embedding_vector = ft.get_word_vector(word)
            if embedding_vector is not None:
                embeddings_matrix[i] = embedding_vector
        except KeyError:
            # Handle the case where the word is not found in FastText model
            continue



In [63]:
model = tf.keras.Sequential([
    # Embedding layer with pre-trained embeddings
    tf.keras.layers.Embedding(
        input_dim=vocab_size,  # Use vocab_size without +1
        output_dim=embedding_dim,
        input_length=max_length,
        weights=[embeddings_matrix],
        trainable=False
    ),
    # Bidirectional LSTM layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.5)),
    # Dense layer with ReLU activation and L2 regularization
    tf.keras.layers.Dense(24, activation='relu', kernel_regularizer=l2(0.01)),
    # Output layer for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [64]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [65]:
model.build((None, max_length))  # Adjust input shape if needed


In [66]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 120, 100)            │       1,881,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          34,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 24)                  │           1,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,916,733 (7.31 MB)

 Trainable params: 35,633 (139.19 KB)

 Non-trainable params: 1,881,100 (7.18 MB)

In [67]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


In [68]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

num_epochs = 100
history = model.fit(
    padded_sequences_train,  # Training data
    labels_train,            # Training labels
    epochs=num_epochs,       # Number of epochs
    batch_size=32,           # Batch size
    validation_split=0.1,   # Fraction of training data to use as validation data
    callbacks=[early_stopping, checkpoint]  # Callbacks to monitor training
)


Epoch 1/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 23s 103ms/step - accuracy: 0.6605 - loss: 0.6815 - val_accuracy: 0.7797 - val_loss: 0.4838
Epoch 2/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 104ms/step - accuracy: 0.7743 - loss: 0.4896 - val_accuracy: 0.7846 - val_loss: 0.5078
Epoch 3/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 100ms/step - accuracy: 0.7715 - loss: 0.4804 - val_accuracy: 0.8087 - val_loss: 0.4383
Epoch 4/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - accuracy: 0.7875 - loss: 0.4687 - val_accuracy: 0.8119 - val_loss: 0.4149
Epoch 5/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.7990 - loss: 0.4407 - val_accuracy: 0.8039 - val_loss: 0.4378
Epoch 6/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - accuracy: 0.8017 - loss: 0.4296 - val_accuracy: 0.8183 - val_loss: 0.4227
Epoch 7/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - accuracy: 0.7999 - loss: 0.4396 - val_accuracy: 0.8264 - val_loss: 0.4277
Epoch 8/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.8206 - lo

In [69]:
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [70]:
result = model.evaluate(testing_padded, labels_test, batch_size=32)
print(f"Test Loss: {result[0]}")
print(f"Test Accuracy: {result[1]}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.8785 - loss: 0.3130 - precision_3: 0.9006 - recall_3: 0.8766
Test Loss: 0.330152690410614
Test Accuracy: 0.8596237301826477


In [71]:
!pip install keras-tuner --upgrade


In [72]:
import keras_tuner as kt

def build_model(hp):
    model = tf.keras.Sequential()

    # Embedding layer with pretrained embeddings
    model.add(tf.keras.layers.Embedding(vocab_size+1,
                                        embedding_dim,
                                        input_length=max_length,
                                        weights=[embeddings_matrix],
                                        trainable=False))

    # Bidirectional LSTM layer
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=hp.Int('units', min_value=16, max_value=128, step=16),
        dropout=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
    )))

    # Dense layer
    model.add(tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=16, max_value=128, step=16),
        activation='relu',
        kernel_regularizer=l2(0.01)))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model


In [73]:
vocab_size = embeddings_matrix.shape[0] - 1


In [74]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size+1,
                              output_dim=embedding_dim,
                              input_length=max_length,
                              weights=[embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.5)),
    tf.keras.layers.Dense(24, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [75]:
tuner_random_search= kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hate_speech_detection'
)

tuner_random_search.search(padded, labels_train,
             epochs=10,
             validation_data=(testing_padded, labels_test),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])



Reloading Tuner from my_dir/hate_speech_detection/tuner0.json


In [77]:
best_model = tuner_random_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [78]:
# Based on the number of metrics used in model compilation
results = best_model.evaluate(testing_padded, labels_test)

# Extracting specific metrics
test_loss = results[0]
test_accuracy = results[1]
precision = results[2]
recall = results[3]

# Printing the results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')



22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.8655 - loss: 0.3449 - precision: 0.8935 - recall: 0.8589
Test Loss: 0.33880487084388733
Test Accuracy: 0.8726483583450317
Precision: 0.9048991203308105
Recall: 0.8509485125541687


In [79]:
tuner_bayesian_optimization = kt.BayesianOptimization(
    build_model,  # model function
    objective='val_accuracy',  # The metric to optimize
    max_trials=20,  # Number of different models to try
    executions_per_trial=1,  # Number of times to evaluate each model
    directory='my_dir',  # Directory to save results
    project_name='my_bayesian_optimization'  # Project name
)

tuner_bayesian_optimization.search(
    padded,                # Training data
    labels_train,          # Training labels
    epochs=20,             # Number of epochs for each trial
    validation_data=(testing_padded, labels_test),  # Validation data
    callbacks=[early_stopping]  # Early stopping to prevent overfitting
)



Trial 20 Complete [00h 04m 16s]
val_accuracy: 0.8581765294075012

Best val_accuracy So Far: 0.8740954995155334
Total elapsed time: 02h 05m 43s


In [80]:

# Best models after tuning
best_model_random_search = tuner_random_search.get_best_models(num_models=1)[0]
best_model_bayesian_optimization = tuner_bayesian_optimization.get_best_models(num_models=1)[0]

# Get predictions from both models
preds_model_random_search = best_model_random_search.predict(testing_padded)
preds_model_bayesian_optimization = best_model_bayesian_optimization.predict(testing_padded)

# Define the weights for each model
weight_random_search = 0.6  # Higher weight for the better-performing model
weight_bayesian_optimization = 0.4  # Lower weight for the other model

# Perform weighted averaging of predictions
ensemble_preds = (weight_random_search * preds_model_random_search) + (weight_bayesian_optimization * preds_model_bayesian_optimization)

# Threshold the probabilities to get final predictions
final_preds = (ensemble_preds > 0.5).astype(int)

# Evaluate the ensemble model
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(labels_test, final_preds)
precision = precision_score(labels_test, final_preds)
recall = recall_score(labels_test, final_preds)

print(f'Ensemble Model Accuracy: {accuracy}')
print(f'Ensemble Model Precision: {precision}')
print(f'Ensemble Model Recall: {recall}')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 157ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step
Ensemble Model Accuracy: 0.8712011577424024
Ensemble Model Precision: 0.8910614525139665
Ensemble Model Recall: 0.8644986449864499
